### Importing Packages

In [1]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4
from netCDF4 import Dataset
import numpy  as np

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator, StrMethodFormatter
from matplotlib.pyplot import cm
from matplotlib.lines import Line2D

from gsw import p_from_z
from gsw import pot_rho_t_exact as prho

import seaborn as sns
# sns.reset_defaults()
sns.set_theme(style='darkgrid')
# sns.set_context('talk')

import datetime

import scipy
import scipy.io
from scipy import interpolate, ndimage, stats
from PIL import Image
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import sys
import subprocess

from datetime import datetime, date

import glob
import os

import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath("D:/00_Masters/00_Work/"))
from Utils.inertP import *
from Utils.veldire2uv import *
from Utils.HistDir import *
from Utils.uv2veldire import *
from Utils.rmse import *

In [ ]:
sns.set_theme(style='darkgrid')

### Loading database

##### Informacoes adicionais

In [ ]:
# Coast_angle_Corr = [-15,0,20,40,40,45] # correção para estação de salvador baseado na literatura, nao no grau calculado 
# names = ['SBNT','SBJP','SBRF','SBMO','SBAR','SBSV']   
# Metar_describ = pd.DataFrame({'label':['SBNT','SBJP','SBRF','SBMO','SBAR','SBSV'],
#                       'Latitude':[-5.91,-7.15,-8.13,-9.51,-10.98,-12.91],
#                       'Longitude': [-35.25,-34.97,-34.92,-35.79,-37.07,-38.33]})

##### Dataset do ERA5

In [ ]:
dsmerged = xr.open_dataset('D:/00_Masters/01_Dados/ERA5/ERA5.nc')
dsmerged = dsmerged.sel(time=slice('2014-11-25', '2019-12-31')) #Slice do tempo para validação do dados

# era_SBNT = dsmerged.sel(latitude=-5.8, longitude=-35.1, method='nearest') # 2º point-5.8 -34.85
# uE_SBNT = era_SBNT['u10']
# vN_SBNT = era_SBNT['v10']

# era_SBJP = dsmerged.sel(latitude=-7.05, longitude=-34.6, method='nearest') # 2º point-7.05 -34.35
# uE_SBJP = era_SBJP['u10']
# vN_SBJP = era_SBJP['v10']

# era_SBRF = dsmerged.sel(latitude=-8.05, longitude=-34.6, method='nearest') # 2º point-8.05 -34.35
# uE_SBRF = era_SBRF['u10']
# vN_SBRF = era_SBRF['v10']

# era_SBMO = dsmerged.sel(latitude=-9.55, longitude=-35.35, method='nearest') # 2º point-9.55 -35.1
# uE_SBMO = era_SBMO['u10']
# vN_SBMO = era_SBMO['v10']

# era_SBAR = dsmerged.sel(latitude=-11.05, longitude=-36.85, method='nearest') # 2º point -11.05 -36.6
# uE_SBAR = era_SBAR['u10']
# vN_SBAR = era_SBAR['v10']

era_SBSV = dsmerged.sel(latitude=-13.05, longitude=-38.35, method='nearest') # 2º point -12.8/-13.05 -37.85/-38.1
uE_SBSV = era_SBSV['u10']
vN_SBSV = era_SBSV['v10']

##### Dataset do CCMP

        Preparar loop para leitura de diversos pontos
        tratamento dos dados
        

In [2]:
mat = scipy.io.loadmat('d:/00_Masters/01_Dados/CCMP_PF_1998_2022.mat')

# mat['CCMP_vE_vN_oceanografico']
# Correção em relaçao a costa feita anteriormente (40º de correcao!) utilizar CCMP_u_v
# para provar uE, vN = map(list, zip(*mat['CCMP_u_v'])); velC, dirC = uv2veldire(-uE,-vN)

# time = pd.DataFrame(mat['time_CCMP_g'],columns=['year','month','day','hour','min','sec'])
time = pd.date_range(start='1988-02-28 00:00:00',end='2022-04-26 18:00:00',freq='3h')
u, v = map(list, zip(*mat['CCMP_u_v']))
u = pd.DataFrame(u,columns=['u']).set_index(time)
v = pd.DataFrame(v,columns=['v']).set_index(time)
velC, dirC = uv2veldire(-u['u'],-v['v'])
velC = pd.DataFrame(velC,columns=['speed']).set_index(time)
dirC = pd.DataFrame(dirC,columns=['direction']).set_index(time)

CCMP = pd.concat([velC, dirC ,u, v], axis=1, join="inner")
CCMP = CCMP['2014-11-25': '2019-12-31']

##### Dataset do METAR

In [ ]:
Metar_describ = pd.DataFrame({'label':['SBSV'],
                      'Latitude':[-12.91],
                      'Longitude': [-38.33]})


i = [0,1,2,3,4,5]
df_list =[]

for n in range(len(names)):
    df = pd.read_csv("D:/00_Masters/01_Dados/METAR/"+ names[n] +".csv",sep=',',
    index_col='times', parse_dates=True)
    df = df['2014-11-25':'2019-12-31'] # Slice do tempo para validação dos dados
    df_list.append(df)

## Analises

#### Calculo do período inericial e filtragem

In [3]:
lat = [-13.05]
Coast_angle_Corr = [45]
names = ['SBSV']   

P = []
for i in lat:
    P.append(inertP(lat=i)) #MEtar_describ

            ERA5

In [ ]:
era = {}

for n,fi,c in zip(names,P,Coast_angle_Corr):
    varU = eval('uE_{}'.format(n)).to_dataframe()
    varU = varU.rolling(str((fi*-1//24)+1)+'D',center=True).mean()
    varU = varU.resample('1D').mean()

    varN = eval('vN_{}'.format(n)).to_dataframe()
    varN = varN.rolling(str((fi*-1//24)+1)+'D',center=True).mean()
    varN = varN.resample('1D').mean()

    erat = varU
    erat['v10'] = varN['v10']
    erat['speed'], erat['direction'] = uv2veldire(erat['u10'],erat['v10'],corr_val=c)#
    erat['u'], erat['v'] = veldire2uv(erat['speed'], erat['direction']) #Replica a coluna para caso haja correção em relação a costa e ajusta nome da coluna
    
    #transformando o vento para convenção oceanográfica (Vento direcionado para onde está indo - semelhante a CORRENTE)
    erat['direction'] -= 180 
    erat['direction'][erat['direction']<0] += 360

    era['era_{}'.format(n)] = erat


            CCMP

In [4]:
ccmp = {}

for i,n,fi,c in zip(range(0,len(names)),names,P,Coast_angle_Corr):
    ccmpTemp = CCMP.rolling(str((fi*-1//24)+1)+'D',center=True).mean()
    ccmpTemp = ccmpTemp.resample('1D').mean()

    ccmp['ccmp_{}'.format(n)] = ccmpTemp

            METAR

In [ ]:
met = {}

for i,n,fi,c in zip(range(0,len(names)),names,P,Coast_angle_Corr):
    ban = df_list[i].rolling(str((fi*-1//24)+1)+'D',center=True).mean()
    ban = ban.resample('1D').mean()
    ban['direction'] = ban['direction'] - c
    ban['u'],ban['v'] = veldire2uv(ban['speed'],ban['direction'])
    ban = ban.interpolate(method='linear')
    ban['direction'] -=180
    ban['direction'][ban['direction']<0] += 360

    met['{}'.format(n)] = ban

#### FIles e Figs - Estatísticas Básicas e Correlação

            Estatísticas e Correlaçoes para periodo total

In [ ]:
vars = ['speed','direction','u','v']
nam = ['ERA5', 'CCMP']
path_output = 'D:/00_Masters/02_Resultados_Parciais/Validacao_Meteo/'


for n in names:
    schema = [era['era_{}'.format(n)], ccmp['ccmp_{}'.format(n)]]
    f1 = open(os.path.join(path_output, './{}/'.format(n),'./Correlacoes_STATs/CCMP_Stats_WND_{}.txt'.format(n)) , 'w')
    print('\n\t',n,file=f1)

    for v in vars:
        for i in range(0,len(schema)):
            print('\n',nam[i],' - Variable: ',v,file=f1)
            print(stats.describe(schema[i][v],nan_policy='omit'), ('std= '+str(np.std(schema[i][v]))),file=f1)
    f1.close()

    f2 = open(os.path.join(path_output,  './{}/'.format(n),'./Correlacoes_STATs/CCMP_R2_{}.txt'.format(n)) , 'w')
    R = []
    RMSE = []
    for v in vars:
        for i in range(0,len(schema)-1):
            res = stats.pearsonr(x=schema[i+1][v],y=schema[0][v])
            err = rmse(schema[i+1][v],schema[0][v])
            print('\n',' - Variable: ',v,file=f2)
            print('Pearson Corr. -', nam[i+1], nam[0],file=f2)
            print(res,file=f2)
            print('Root Mean Square Error RMSE -', nam[i+1], nam[0],file=f2)
            print(err,file=f2)

            R.append(np.round(res[0],2))
            RMSE.append(np.round(err,2))
    f2.close()

    ''' 
    FIGURA COM SeaBorn

    '''
    for ind,v in enumerate(vars):
        fig, [ax1,ax2] = plt.subplots(nrows=1, ncols=2, gridspec_kw={'width_ratios':[2,2], 'height_ratios':[1]})
        cmap = sns.color_palette(as_cmap=True)

        with sns.axes_style("darkgrid"):
            for i in range(0,len(schema)):
                ax1 = sns.kdeplot(data=schema[i][v], x = schema[i][v], fill=True, alpha=.2, linewidth=2, bw=0.2, ax=ax1)

        with sns.axes_style("darkgrid"):
            for i in range(0,len(schema)-1):
                ax2 = sns.regplot(x=schema[i+1][v], y=schema[0][v], color=cmap[i+1], fit_reg = False, ax=ax2)

        fig.subplots_adjust(wspace=0.5)
        fig.suptitle('{}'.format(n)+' - '+v.format().capitalize(),fontsize=14,y=.95)

        if str(v) == 'speed':
            unit = "[m.s$^{-1}$]"
            ax1.set_xticks(np.arange(0,12,2.5))
            ax2.set_xticks(np.arange(0,12,2.5))
            ax2.set_yticks(np.arange(0,12,2.5))
            ax1.set_ylim(0,.7)
        elif str(v) == 'direction':
            unit = "[°]"
            ax1.set_xticks(np.arange(0,360,90))
            ax2.set_xticks(np.arange(0,360,90))
            ax2.set_yticks(np.arange(0,360,90))
            ax1.set_ylim(0,0.04)
        else:
            unit = ""
            ax1.set_xticks(np.arange(-8,10,2))
            ax2.set_xticks(np.arange(-8,10,2))
            ax2.set_yticks(np.arange(-8,10,2))
            ax1.set_ylim(0,.56)

        ax1.set_xlabel(unit)
        ax2.set_xlabel(nam[1] + ' ' + unit)
        ax2.set_ylabel(nam[0] + ' ' + unit)
        
        ax1.legend(nam,fontsize=12, bbox_to_anchor = (-.2,-.18),loc='upper left', ncol=3)

        # ax2.annotate('Pearson. Coef.:', fontsize=6.5,color='k',xy=(.6,.15),xycoords='axes fraction')
        ax2.annotate("$r^2$ = " + str(R[ind]), fontsize=6.5,color=cmap[1],
                    xy=(.6, .1), xycoords='axes fraction',fontweight="bold")
        ax2.annotate("RMSE = " + str(RMSE[ind]), fontsize=6.5,color=cmap[1],
                    xy=(.6, .065), xycoords='axes fraction',fontweight="bold")
        

        plt.savefig('D:/00_Masters/02_Resultados_Parciais/Validacao_Meteo/{}/Correlacoes_STATs/CCMP_{}_{}.png'.format(n,n,v), format='png', dpi=300, bbox_inches='tight')

        plt.close()

CCMP X METAR

In [ ]:
vars = ['speed','direction','u','v']
nam = ['Metar','CCMP']
path_output = 'D:/00_Masters/02_Resultados_Parciais/Validacao_Meteo/'


for n in names:
    schema = [met[n], ccmp['ccmp_{}'.format(n)],]
    f1 = open(os.path.join(path_output, './{}/'.format(n),'./Correlacoes_STATs/CCMP-Met_Stats_WND_{}.txt'.format(n)) , 'w')
    print('\n\t',n,file=f1)

    for v in vars:
        for i in range(0,len(schema)):
            print('\n',nam[i],' - Variable: ',v,file=f1)
            print(stats.describe(schema[i][v],nan_policy='omit'), ('std= '+str(np.std(schema[i][v]))),file=f1)
    f1.close()

    f2 = open(os.path.join(path_output,  './{}/'.format(n),'./Correlacoes_STATs/CCMP-Met_R2_{}.txt'.format(n)) , 'w')
    R = []
    RMSE = []
    for v in vars:
        for i in range(0,len(schema)-1):
            res = stats.pearsonr(x=schema[i+1][v],y=schema[0][v])
            err = rmse(schema[i+1][v],schema[0][v])
            print('\n',' - Variable: ',v,file=f2)
            print('Pearson Corr. -', nam[i+1], nam[0],file=f2)
            print(res,file=f2)
            print('Root Mean Square Error RMSE -', nam[i+1], nam[0],file=f2)
            print(err,file=f2)

            R.append(np.round(res[0],2))
            RMSE.append(np.round(err,2))
    f2.close()

    ''' 
    FIGURA COM SeaBorn

    '''
    for ind,v in enumerate(vars):
        fig, [ax1,ax2] = plt.subplots(nrows=1, ncols=2, gridspec_kw={'width_ratios':[2,2], 'height_ratios':[1]})
        cmap = sns.color_palette(as_cmap=True)

        with sns.axes_style("darkgrid"):
            for i,c in zip(range(0,len(schema)),[3,1]):
                ax1 = sns.kdeplot(data=schema[i][v], x = schema[i][v], color=cmap[c], fill=True, alpha=.2, linewidth=2, bw=0.2, ax=ax1)

        with sns.axes_style("darkgrid"):
            for i in range(0,len(schema)-1):
                ax2 = sns.regplot(x=schema[i+1][v], y=schema[0][v], color=cmap[i+1], fit_reg = False, ax=ax2)

        fig.subplots_adjust(wspace=0.5)
        fig.suptitle('{}'.format(n)+' - '+v.format().capitalize(),fontsize=14,y=.95)

        if str(v) == 'speed':
            unit = "[m.s$^{-1}$]"
            ax1.set_xticks(np.arange(0,12,2.5))
            ax2.set_xticks(np.arange(0,12,2.5))
            ax2.set_yticks(np.arange(0,12,2.5))
            ax1.set_ylim(0,.7)
        elif str(v) == 'direction':
            unit = "[°]"
            ax1.set_xticks(np.arange(0,360,90))
            ax2.set_xticks(np.arange(0,360,90))
            ax2.set_yticks(np.arange(0,360,90))
            ax1.set_ylim(0,0.04)
        else:
            unit = ""
            ax1.set_xticks(np.arange(-8,10,2))
            ax2.set_xticks(np.arange(-8,10,2))
            ax2.set_yticks(np.arange(-8,10,2))
            ax1.set_ylim(0,.56)

        ax1.set_xlabel(unit)
        ax2.set_xlabel(nam[1] + ' ' + unit)
        ax2.set_ylabel(nam[0] + ' ' + unit)
        
        ax1.legend(nam,fontsize=12, bbox_to_anchor = (-.2,-.18),loc='upper left', ncol=3)

        # ax2.annotate('Pearson. Coef.:', fontsize=6.5,color='k',xy=(.6,.15),xycoords='axes fraction')
        ax2.annotate("$r^2$ = " + str(R[ind]), fontsize=6.5,color=cmap[1],
                    xy=(.6, .1), xycoords='axes fraction',fontweight="bold")
        ax2.annotate("RMSE = " + str(RMSE[ind]), fontsize=6.5,color=cmap[1],
                    xy=(.6, .065), xycoords='axes fraction',fontweight="bold")
        

        plt.savefig('D:/00_Masters/02_Resultados_Parciais/Validacao_Meteo/{}/Correlacoes_STATs/CCMP-Met_{}_{}.png'.format(n,n,v), format='png', dpi=300, bbox_inches='tight')

        plt.close()

In [ ]:
Season = [[1,2,3],[4,5,6],[4,5,6],[10,11,12]] #[[12,1,2],[3,4,5],[6,7,8],[9,10,11]] 
Season_names = ['SUMMER','AUTUMN','WINTER','SPRING']

            SEASONAL Estatísticas e Correlaçoes
                Season = [[1,2,3],[4,5,6],[4,5,6],[10,11,12]]

In [ ]:
for n in names:
    for s,sn in zip(Season,Season_names):
        ccmp_season = ccmp['ccmp_{}'.format(n)]
        ccmp_season = ccmp_season[ccmp_season.index.month.isin(s)]
        era_season = era['era_{}'.format(n)]
        era_season = era_season[era_season.index.month.isin(s)]
        
        schema = [ccmp_season, era_season]

        f1 = open(os.path.join(path_output, './{}/'.format(n),'./Correlacoes_STATs/CCMP_Stats_WND_{}_{}.txt'.format(n,sn)) , 'w')
        print('\n\t',n,sn,file=f1)

        for v in vars:
            for i in range(0,len(schema)):
                print('\n',nam[i],' - Variable: ',v,file=f1)
                print(stats.describe(schema[i][v],nan_policy='omit'), ('std= '+str(np.std(schema[i][v]))),file=f1)
        f1.close()

        f2 = open(os.path.join(path_output, './{}/'.format(n),'./Correlacoes_STATs/CCMP_R2_{}_{}.txt'.format(n,sn)) , 'w')
        R = []
        RMSE = []
        for v in vars:
            for i in range(0,len(schema)-1):
                res = stats.pearsonr(x=schema[i+1][v],y=schema[0][v])
                err = rmse(schema[i+1][v],schema[0][v])
                print('\n',' - Variable: ',v,file=f2)
                print('\nPearson Corr. -', nam[i+1], nam[0],file=f2)
                print(res,file=f2)

                R.append(np.round(res[0],5))
                RMSE.append(np.round(err,3))                
        f2.close()

        ''' 
        FIGURA COM SeaBorn

        '''
        for v in vars:
            fig, [ax1,ax2] = plt.subplots(nrows=1, ncols=2, gridspec_kw={'width_ratios':[2,2], 'height_ratios':[1]})
            cmap = sns.color_palette(as_cmap=True)

            with sns.axes_style("darkgrid"):
                for i in range(0,len(schema)):
                    ax1 = sns.kdeplot(data=schema[i][v], x = schema[i][v], fill=True, alpha=.2, linewidth=2, bw=0.2, ax=ax1)

            with sns.axes_style("darkgrid"):
                for i in range(0,len(schema)-1):
                    ax2 = sns.regplot(x=schema[i+1][v], y=schema[0][v], color=cmap[i+1], fit_reg = False, ax=ax2)

            fig.subplots_adjust(wspace=0.4)
            fig.suptitle('{}-{}'.format(n,sn)+' - '+v,fontsize=14,y=.95)

            if str(v) == 'speed':
                unit = "[m.s$^{-1}$]"
                ax1.set_xticks(np.arange(0,12,2.5))
                ax2.set_xticks(np.arange(0,12,2.5))
                ax2.set_yticks(np.arange(0,12,2.5))
                ax1.set_ylim(0,1)
            elif str(v) == 'direction':
                unit = "[°]"
                ax1.set_xticks(np.arange(0,360,90))
                ax2.set_xticks(np.arange(0,360,90))
                ax2.set_yticks(np.arange(0,360,90))
                ax1.set_ylim(0,0.08)
            else:
                unit = ""
                ax1.set_xticks(np.arange(-8,10,2))
                ax2.set_xticks(np.arange(-8,10,2))
                ax2.set_yticks(np.arange(-8,10,2))
                ax1.set_ylim(0,1)

            ax1.set_xlabel(unit)
            ax2.set_xlabel('ERA5 ' + unit)
            ax2.set_ylabel('CCMP ' + unit)
            
            ax1.legend(nam,fontsize=12, bbox_to_anchor = (-.2,-.18),loc='upper left', ncol=3)

            ax2.annotate('Pearson. Coef.:', fontsize=6.5,color='k',xy=(.6,.15),xycoords='axes fraction')
            ax2.annotate("$r^2$ = " + str(R[ind]), fontsize=6.5,color=cmap[1],
                        xy=(.6, .1), xycoords='axes fraction',fontweight="bold")
            ax2.annotate("RMSE = " + str(RMSE[ind]), fontsize=6.5,color=cmap[1],
                        xy=(.6, .065), xycoords='axes fraction',fontweight="bold")

            plt.savefig('D:/00_Masters/02_Resultados_Parciais/Validacao_Meteo/{}/Correlacoes_STATs/CCMP_{}_{}_{}.png'.format(n,n,v,sn), format='png', dpi=300, bbox_inches='tight')

#### Serie Temporal

In [ ]:
vars = ['speed','direction','u','v']
nam = ['CCMP']
path_output = 'D:/00_Masters/02_Resultados_Parciais/Validacao_Meteo/'

for n in names:
    schema = [ccmp['ccmp_{}'.format(n)],]

    for v in vars:
        fig, ax1 = plt.subplots(nrows=1, ncols=1, gridspec_kw={'width_ratios':[2], 'height_ratios':[1]})
        cmap = sns.color_palette(as_cmap=True)
        fig.suptitle('{}'.format(n).upper(),fontsize=14,y=.95)

        with sns.axes_style("darkgrid"):
            for i in range(0,len(schema)):
                ax1 = sns.lineplot(x=schema[i][v].index,y=schema[i][v],ax=ax1,color=cmap[1])

        ax1.autoscale(enable=True, axis='x', tight=True)
        ax1.legend(nam,fontsize=12, bbox_to_anchor = (0.35,-.14),loc='upper left', ncol=3)
        ax1.set_xlabel('Year')
        if str(v) == 'speed':
                    unit = "[m.s$^{-1}$]"
        elif str(v) == 'direction':
                    unit = "[°]"
        else:
            unit = ""
        ax1.set_ylabel('{}'.format(v).capitalize() + ' ' +unit)

        plt.savefig('D:/00_Masters/02_Resultados_Parciais/Validacao_Meteo/{}/Serie_Temporal/CCMP_Serie_{}_{}.png'.format(n,n,v), format='png', dpi=300, bbox_inches='tight')

#### HistDir

refazendo dataset sem a correção para convenção oceanográfica do vento
pois a rotina HistDir já faz automaticamente essa conversão;

            ERA5

In [ ]:
era = {}

for n,fi,c in zip(names,P,Coast_angle_Corr):
    varU = eval('uE_{}'.format(n)).to_dataframe()
    varU = varU.rolling(str((fi*-1//24)+1)+'D',center=True).mean()
    varU = varU.resample('1D').mean()

    varN = eval('vN_{}'.format(n)).to_dataframe()
    varN = varN.rolling(str((fi*-1//24)+1)+'D',center=True).mean()
    varN = varN.resample('1D').mean()

    erat = varU
    erat['v10'] = varN['v10']
    erat['speed'], erat['direction'] = uv2veldire(erat['u10'],erat['v10'],corr_val=c)#
    erat['u'], erat['v'] = veldire2uv(erat['speed'], erat['direction']) #Replica a coluna para caso haja correção em relação a costa e ajusta nome da coluna
    
    erat['direction'][erat['direction']<0] += 360

    era['era_{}'.format(n)] = erat


            CCMP

                     -- CCMP está corrigido para convenção oceanográfica, ou seja, precisamos criar o vetor inverso

In [5]:
ccmp = {}

for i,n,fi,c in zip(range(0,len(names)),names,P,Coast_angle_Corr):
    ccmpTemp = CCMP.rolling(str((fi*-1//24)+1)+'D',center=True).mean()
    ccmpTemp = ccmpTemp.resample('1D').mean()

    ccmpTemp['direction'] -= 180 
    ccmpTemp['direction'][ccmpTemp['direction']<0] += 360

    ccmp['ccmp_{}'.format(n)] = ccmpTemp

In [7]:
vars = ['speed','direction']
nam = ['CCMP']
for n in names:
    schema = [ccmp['ccmp_{}'.format(n)]]

    for i in range(0,len(schema)):
        outputname = 'D:/00_Masters/02_Resultados_Parciais/Validacao_Meteo/{}/HistDir/'.format(n)+str(nam[i])+'_'+str(n)
        vel = schema[i]['speed']
        dire = schema[i]['direction']
        
        HistDir(P=vel,D=dire,Pmax=10,MaxProb=80,
                arqname=outputname,par='wind',interpolado=False, conv_oc=True)

        for s,sn in zip(Season,Season_names):
            
            schema_S = schema[i]
            schema_S = schema_S[schema_S.index.month.isin(s)]

            outputname = 'D:/00_Masters/02_Resultados_Parciais/Validacao_Meteo/{}/HistDir/'.format(n)+str(n)+'_'+str(nam[i])+'_'+'_'+sn
        
            HistDir(P=schema_S['speed'],D=schema_S['direction'],Pmax=10,MaxProb=80,
                    arqname=outputname,
                    par='vento',interpolado=False, conv_oc=True)

            print(str(n)+'_'+str(nam[i])+'_'+'_'+sn)

[ 0.  2.  4.  6.  8. 10.]
Done
